In [7]:
import pandas as pd

In [8]:
#In the future, we can improve this by being able to read in any/all the datasets

#reads in the data set
data = pd.read_csv(r"Affiliation_Parsing\NeurIPS affiliations\NeurIPSaffiliations-2010-affiliation.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                292 non-null    int64 
 1   Unnamed: 0.1              292 non-null    int64 
 2   first-author              292 non-null    object
 3   first-author-affiliation  292 non-null    object
 4   last-author               292 non-null    object
 5   last-author-affiliation   291 non-null    object
 6   title                     292 non-null    object
dtypes: int64(2), object(5)
memory usage: 16.1+ KB


In [9]:
#drops every column that is not the authors or their affiliations
data = data.drop(data.columns[0:2],axis=1)
data = data.drop(data.columns[4:5],axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   first-author              292 non-null    object
 1   first-author-affiliation  292 non-null    object
 2   last-author               292 non-null    object
 3   last-author-affiliation   291 non-null    object
dtypes: object(4)
memory usage: 9.2+ KB


In [10]:
#imports BS4 libraries for webscraping
from re import I
import requests
from bs4 import BeautifulSoup

In [11]:
#Need to add in they/them
#Improve search


def findGender(urlLink):
    gender = ''
    ## put a url where the desciption of the author is avilable
    url = urlLink
    htmlText = requests.get(url).text
    soup = BeautifulSoup(htmlText, 'html.parser')


    # gathers all the p tags from the html
    soupstring = str(soup.find_all("p"))
    shetags = [" she ", " She "," her ", " Her "," hers "," Hers "]
    hetags = [" he ", " He "," his ", " His "," him "," Him "]

    ##checks if male or female pronouns are present in the desciption
    containsfemale = any(femalepronouns in soupstring for femalepronouns in shetags)
    containsmale = any(malepronouns in soupstring for malepronouns in hetags)

    if containsfemale:
        gender = "female"
    elif containsmale:
        gender = "male"
    else:
        gender = "undefined"
    return gender

In [12]:
#returns whether or not a substring exists in a string
def substring_exists(string, sub_string):
    return string.find(sub_string)

In [13]:
#returns a list of the first-authors
def getFirstAuthorList():
    for column in data[['first-author']]:
        authors = data[column]

        '''
        Prints out column name and its values
        '''
        #print('Column Name: '+column)
        #print(authors.values)

    lst_authors = authors.values
    return lst_authors

In [14]:
#imports googlesearch to use requests and BS4 to scrape google
from googlesearch import search

In [17]:
#initializes new column for the gender of the first author
data['first-gender'] = ''

#fills in gender column for first-author column
def genderFirstAuthor():

    #initializes counter that keep track of author index
    counter = 0

    #gets list of the first authors
    lst=getFirstAuthorList()

    #loops through each author in the dataset
    for person in lst:

        '''
        search term for googlesearch
        follows structure of author's name, their affiliation, and .edu
        i.e. Glendon Chin Steven's Institute of Technology .edu
        '''
        query=person+" "+data['first-author-affiliation'].values[counter]+" .edu"


        
        #gets multiple links based on the search term
        for link in search(query, 5, "en", None):

            #if the link is not .edu, we will not use that link
            if(substring_exists(link, ".edu")!=-1):
                data['first-gender'].values[counter]=findGender(link)
                print("Author: "+data['first-author'].values[counter])
                print("Affiliation: "+data['first-author-affiliation'].values[counter])
                print("Gender: "+data['first-gender'].values[counter])
                print("Counter: "+str(counter))
                print("Link: "+link)
                print()
                break
            else:
                print("Skipped "+link)
        print()
        
        #increments index, moves to the next row
        counter+=1


In [18]:
genderFirstAuthor()

Author: Surya Ganguli
Affiliation: Stanford
Gender: undefined
Counter: 0
Link: https://ganguli-gang.stanford.edu/


Skipped https://www.microsoft.com/en-us/research/people/hal3/
Author: Hal Daumé III
Affiliation: Univ of Maryland / Microsoft Research
Gender: male
Counter: 1
Link: http://www.umiacs.umd.edu/~hal/


Skipped https://www.linkedin.com/in/akshat-kumar-7a8250
Author: Akshat Kumar
Affiliation: University of Massachusetts, Amherst
Gender: male
Counter: 2
Link: https://www.cics.umass.edu/news/latest-news/akshat-kumar-wins-icaps-2014-best-dissertation-award


Author: Alan Fern
Affiliation: Oregon State University
Gender: male
Counter: 3
Link: https://eecs.oregonstate.edu/people/fern-alan


Author: Deqing Sun
Affiliation: Brown University
Gender: undefined
Counter: 4
Link: http://cs.brown.edu/~dqsun/research/


Skipped https://zicokolter.com/
Skipped http://zicokolter.com/bosch/
Skipped https://www.bosch-ai.com/research/zico-kolter/
Author: J. Zico Kolter
Affiliation: Carnegie Mell

KeyboardInterrupt: 

In [50]:
#As you can see, we need to look into optimizing the findGender function
#I will continue to work on the googlesearch so that it gives the best websites to navigate
print(data[['first-gender']])

    first-gender
0      undefined
1      undefined
2      undefined
3      undefined
4      undefined
..           ...
287             
288             
289             
290             
291             

[292 rows x 1 columns]


In [1]:
import threading
import time

In [13]:
import concurrent.futures

In [38]:
from random import randint

#skeleton for actual timer function
#this example shows how the timer will function concurrently with the function

def checkTime(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

def runTimer():
    value = randint(0, 5)
    while value!=3:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            secs = [3,2,1]
            results = executor.map(checkTime, secs)
        value = randint(0, 5)
        print(value)

runTimer()

Sleeping 3 second(s)...
Sleeping 2 second(s)...
Sleeping 1 second(s)...
0
Sleeping 3 second(s)...
Sleeping 2 second(s)...
Sleeping 1 second(s)...
2
Sleeping 3 second(s)...Sleeping 2 second(s)...
Sleeping 1 second(s)...

1
Sleeping 3 second(s)...
Sleeping 2 second(s)...
Sleeping 1 second(s)...
3
